In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("pre_sentiment_data_2.csv")
df.head()

,link,headline,category,short_description,authors,date,train_text,train_text_tokens,dominant_topic_id_lda,dominant_topic_prob_lda
0,https://www.huffingtonpost.com/entry/%C3%A9dga...,Édgar Ramírez Stands In Solidarity With Venezu...,LATINO VOICES,“The recall vote (revocatory referendum) is cl...,Tanisha Love Ramirez,2016-09-01,Édgar Ramírez Stands In Solidarity With Venezu...,"['dgar', 'ram', 'rez', 'stand', 'solidarity', ...",4,0.340438
1,https://www.huffingtonpost.com/entry/-20-reaso...,20 Reasons to Drop Everything and Go to Spain,TRAVEL,See more photos of natural parks in Spain 20.)...,"minube, Contributor\nCommunity of over 2 milli...",2014-01-15,20 Reasons to Drop Everything and Go to Spain ...,"['reason', 'drop', 'everything', 'spain', 'see...",4,0.315553
2,https://www.huffingtonpost.com/entry/-2014-bet...,WATCH: 2014 BET Awards Performances,BLACK VOICES,NaN,Brennan Williams,2014-06-30,WATCH: 2014 BET Awards Performances,"['watch', 'bet', 'award', 'performance']",4,0.318119
3,https://www.huffingtonpost.com/entry/-4-not-ou...,#4 Not Our Finest Hour: Why Is Liberal America...,POLITICS,"In countless ways, the political force that ha...","Andy Schmookler, ContributorAward-winning auth...",2014-09-30,#4 Not Our Finest Hour: Why Is Liberal America...,"['finest', 'hour', 'liberal', 'america', 'fall...",4,0.279397
4,https://www.huffingtonpost.com/entry/-5-looks-...,Flat Out Adorable: 5 Looks to Mimic This Season,STYLE,Whether you're out exploring a new city or hea...,"Alisa Gould-Simon, ContributorCo-Founder, Pose",2014-06-19,Flat Out Adorable: 5 Looks to Mimic This Seaso...,"['flat', 'adorable', 'look', 'mimic', 'season'...",1,0.263950


In [3]:
df.shape

(209012, 10)

In [4]:
df['train_text'].isna().sum()

np.int64(0)

In [5]:
from transformers import pipeline

In [6]:
# Initializing the sentiment analysis pipeline using the pre-trained distilBERT model.

# Set batch_size for faster processing of large dataframes.
print("Initializing sentiment analysis pipeline (This may take a moment to download the model)...")
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", device=0)
# If you don't have a GPU or encounter issues, change device=0 to device=-1 for CPU.
# sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", device=-1)

print("Sentiment analysis pipeline initialized.")

Initializing sentiment analysis pipeline (This may take a moment to download the model)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


Sentiment analysis pipeline initialized.


In [7]:
# Applying the sentiment analysis to the 'train_text' column of your DataFrame.
# Transformer models work best on raw, un-tokenized text.
# We'll get a list of dictionaries, each containing 'label' (POSITIVE/NEGATIVE) and 'score'.
print("Performing sentiment analysis on news articles...")

sentiment_results = sentiment_pipeline(df['train_text'].tolist(), truncation=True, batch_size=32)
# truncation=True ensures long texts are truncated to the model's max input length.
# batch_size can be adjusted based on your system's memory.

print("Sentiment analysis complete.")

Performing sentiment analysis on news articles...
Sentiment analysis complete.


In [8]:
# Extracting the sentiment labels and scores into new DataFrame columns.
# Each result is a dictionary like {'label': 'POSITIVE', 'score': 0.999}.
df['sentiment_label'] = [result['label'] for result in sentiment_results]
df['sentiment_score'] = [result['score'] for result in sentiment_results]

print("\nFirst 5 rows with sentiment analysis results:")
df[['headline', 'train_text', 'sentiment_label', 'sentiment_score']].head()


First 5 rows with sentiment analysis results:


,headline,train_text,sentiment_label,sentiment_score
0,Édgar Ramírez Stands In Solidarity With Venezu...,Édgar Ramírez Stands In Solidarity With Venezu...,POSITIVE,0.995566
1,20 Reasons to Drop Everything and Go to Spain,20 Reasons to Drop Everything and Go to Spain ...,POSITIVE,0.906650
2,WATCH: 2014 BET Awards Performances,WATCH: 2014 BET Awards Performances,POSITIVE,0.998318
3,#4 Not Our Finest Hour: Why Is Liberal America...,#4 Not Our Finest Hour: Why Is Liberal America...,NEGATIVE,0.998269
4,Flat Out Adorable: 5 Looks to Mimic This Season,Flat Out Adorable: 5 Looks to Mimic This Seaso...,POSITIVE,0.999155


In [9]:
# Display sentiment distribution
print("\nSentiment Label Distribution:")
print(df['sentiment_label'].value_counts())

print("\nSentiment Score Statistics:")
print(df['sentiment_score'].describe())


Sentiment Label Distribution:
sentiment_label
NEGATIVE    112445
POSITIVE     96567
Name: count, dtype: int64

Sentiment Score Statistics:
count    209012.000000
mean          0.957210
std           0.091692
min           0.500000
25%           0.971433
50%           0.993685
75%           0.998479
max           0.999893
Name: sentiment_score, dtype: float64


* The  dataset has a slight lean towards negative sentiment. This is quite common for news datasets, as news often reports on challenges, conflicts, and problems (e.g., crime, politics, world events).
* Very High Confidence Scores: The mean (0.957), 25% (0.971), 50% (0.993), and 75% (0.998) values for sentiment_score are extremely high. The standard deviation for the scores is also very low, which indicates that the distilBERT model is generally very confident in its sentiment predictions.
* Minimum Score of 0.5: Such a score can exist when there are neutral or ambiguous texts where the model truly cannot lean one way or another, which is also typical of some news categories.

In [10]:
# Deliverable: Distribution of sentiment by topic (using the dominant_topic_id_lda)
print("\nSentiment Distribution by Dominant LDA Topic:")
sentiment_by_topic = df.groupby('dominant_topic_id_lda')['sentiment_label'].value_counts(normalize=True).unstack(fill_value=0)
print(sentiment_by_topic)


Sentiment Distribution by Dominant LDA Topic:
sentiment_label        NEGATIVE  POSITIVE
dominant_topic_id_lda                    
0                      0.425187  0.574813
1                      0.321101  0.678899
2                      0.798398  0.201602
3                      0.340120  0.659880
4                      0.533810  0.466190
5                      0.329131  0.670869
6                      0.749550  0.250450


In [11]:
# We can also analyze mean sentiment score by topic:
if 'dominant_topic_id_lda' in df.columns:
    mean_sentiment_score_by_topic = df.groupby('dominant_topic_id_lda')['sentiment_score'].mean()
    print("\nMean Sentiment Score by Dominant LDA Topic:")
    print(mean_sentiment_score_by_topic)


Mean Sentiment Score by Dominant LDA Topic:
dominant_topic_id_lda
0    0.957298
1    0.964383
2    0.958107
3    0.955183
4    0.957517
5    0.948751
6    0.953109
Name: sentiment_score, dtype: float64


* Topic 0 (0.425 NEG / 0.575 POS): __Family__<br>
Sentiment Alignment: It's moderately positive. This makes sense. While parenting and life can have challenges, a lot of content in these areas (e.g., advice, feel-good stories, tips) tends to be framed positively.

* Topic 1 (0.321 NEG / 0.679 POS): __Travel / Lifestyle__<br>
Sentiment Alignment: It's predominantly positive. This is consistent with content about "best" lists, weddings, travel destinations, and general positive events.

* Topic 2 (0.798 NEG / 0.202 POS): __Crime / U.S. Local News__<br>
Sentiment Alignment: Extremely negative (nearly 80% negative). This perfectly aligns with the topic content. News about deaths, shootings, and large-scale incidents is almost always negative.

* Topic 3 (0.340 NEG / 0.660 POS): __Politics / World News__<br>
Sentiment Alignment: Predominantly positive (66% positive). This is surprising given the "Ukraine," "Russia," "protest" words, which are often associated with negative sentiment. It might indicate that many articles assigned to this topic related to, for example, solidarity with Ukraine, positive diplomatic efforts, or positive cultural aspects related to these regions, rather than just conflict. Or the "food" and "art" elements pull it up. This topic would be worth deeper manual inspection.

* Topic 4 (0.534 NEG / 0.466 POS): __General Interests__<br>
Sentiment Alignment: Slightly negative (53% negative). Given its generic nature, it's not surprising it hovers around 50/50, perhaps absorbing some of the general negativity prevalent in news.

* Topic 5 (0.329 NEG / 0.671 POS): __Entertainment / Media__<br>
Sentiment Alignment: Predominantly positive. This makes perfect sense for entertainment reviews, celebrity news, and media features.

* Topic 6 (0.750 NEG / 0.250 POS): __Donald Trump / U.S. Republican Party__<br>
Sentiment Alignment: Very negative (75% negative). This is a very strong and expected correlation, reflecting the often critical or contentious nature of reporting around this political figure and related events.

In [12]:
df.shape

(209012, 12)

In [13]:
df.to_csv("post_sentiment_data_2.csv", index=False)

## Trend Visualization

In [15]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [16]:
# --- Step 1: We must ensure 'date' column is in datetime format and create a 'month_year' column ---
# It's crucial for time-series plotting and aggregation.
# The 'date' column is often read as an object/string. We Convert it.
df['date'] = pd.to_datetime(df['date'])

# Create a 'month_year' column for monthly aggregation (e.g., '2014-01')
df['month_year'] = df['date'].dt.to_period('M').astype(str)

print(df[['date', 'month_year']].head())

        date month_year
0 2016-09-01    2016-09
1 2014-01-15    2014-01
2 2014-06-30    2014-06
3 2014-09-30    2014-09
4 2014-06-19    2014-06


In [17]:
# --- Step 2: We prepare data for Overall Sentiment Trend Plot ---
# Group by 'month_year' and 'sentiment_label', then count occurrences.
sentiment_over_time = df.groupby(['month_year', 'sentiment_label']).size().unstack(fill_value=0)

# Calculate proportions for plotting
sentiment_over_time_prop = sentiment_over_time.div(sentiment_over_time.sum(axis=1), axis=0)
sentiment_over_time_prop = sentiment_over_time_prop.reset_index()

# Melt the DataFrame to a 'long' format suitable for Plotly Express
sentiment_over_time_long = sentiment_over_time_prop.melt(
    id_vars=['month_year'],
    value_vars=['POSITIVE', 'NEGATIVE'],
    var_name='Sentiment',
    value_name='Proportion'
)

# Convert 'month_year' back to datetime for correct sorting on plot
sentiment_over_time_long['month_year_dt'] = pd.to_datetime(sentiment_over_time_long['month_year'])
sentiment_over_time_long = sentiment_over_time_long.sort_values('month_year_dt')

In [19]:
# --- Step 3: Plot Overall Sentiment Trend (Proportion of Positive/Negative over time) ---
print("Generating Overall Sentiment Trend Plot...")
fig_sentiment = px.line(
    sentiment_over_time_long,
    x='month_year_dt',
    y='Proportion',
    color='Sentiment',
    title='Overall News Sentiment Trend Over Time (Monthly Proportion)',
    labels={'month_year_dt': 'Date', 'Proportion': 'Proportion of Articles'},
    line_shape='spline', # Smooth the lines
    hover_name='Sentiment',
    hover_data={'Proportion': ':.2%'} # Format proportion in hover
)

fig_sentiment.update_layout(
    xaxis_title="Date",
    yaxis_title="Proportion",
    legend_title="Sentiment",
    hovermode="x unified" # Shows all data for a specific x-value on hover
)
fig_sentiment.show()

Generating Overall Sentiment Trend Plot...


In [20]:
# --- Step 4: Prepare data for Topic Popularity Trend Plot ---
# Group by 'month_year' and 'dominant_topic_id_lda', then count occurrences.
topic_over_time = df.groupby(['month_year', 'dominant_topic_id_lda']).size().unstack(fill_value=0)

# Calculate proportions for plotting
topic_over_time_prop = topic_over_time.div(topic_over_time.sum(axis=1), axis=0)
topic_over_time_prop = topic_over_time_prop.reset_index()

# Rename topic columns for better legend readability if you have defined topic names
# For example: topic_over_time_prop.rename(columns={0: 'Topic_Parenting', 1: 'Topic_Lifestyle', ...}, inplace=True)
# For now, we'll keep the numeric IDs, but in a dashboard you might map these to descriptive names.

# Melt to long format
topic_over_time_long = topic_over_time_prop.melt(
    id_vars=['month_year'],
    var_name='Topic ID',
    value_name='Proportion'
)

topic_over_time_long['month_year_dt'] = pd.to_datetime(topic_over_time_long['month_year'])
topic_over_time_long = topic_over_time_long.sort_values('month_year_dt')

In [34]:
# Create a mapping dictionary for topic names
topic_names = {
    0: "Family",
    1: "Travel/ Lifestyle",
    2: "Crime / U.S. Local News",
    3: "Politics/ World News",
    4: "General Interests",
    5: "Entertainment",
    6: "Donald Trump"
}

# Apply the mapping to your dataframe
# Assuming your 'Topic ID' column contains integers
topic_over_time_long['Topic Name'] = topic_over_time_long['Topic ID'].map(topic_names)

# --- Step 5: Plot Topic Popularity Trend (Proportion of each topic over time) ---
print("Generating Topic Popularity Trend Plot...")
fig_topic_proportion = px.area(
    topic_over_time_long,
    x='month_year_dt',
    y='Proportion',
    color='Topic Name',  # Changed from 'Topic ID' to 'Topic Name'
    title='Dominant Topic Proportion Over Time (Monthly)',
    labels={'month_year_dt': 'Date', 'Proportion': 'Proportion of Articles', 'Topic Name': 'Topic'},
    line_shape='spline',
    hover_name='Topic Name',  # Changed from 'Topic ID' to 'Topic Name'
    hover_data={'Proportion': ':.2%'}
)

fig_topic_proportion.update_layout(
    xaxis_title="Date",
    yaxis_title="Proportion",
    legend_title="Topic",  # Changed from "Topic ID" to "Topic"
    hovermode="x unified"
)

fig_topic_proportion.show()

Generating Topic Popularity Trend Plot...
